In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras import models, layers

In [2]:
teams = pd.read_csv('data.csv')

In [3]:
teams.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [4]:
# Select relevant columns
teams = teams[['NOC', 'Team', 'Year', 'Name', 'Age', 'Medal']]

In [5]:
# Drop rows with missing values for simplicity
teams = teams.dropna()

In [6]:
# Feature engineering
le = LabelEncoder()
teams['NOC'] = le.fit_transform(teams['NOC'])

In [7]:
# Select features and target variable
features = ['Age', 'NOC']
target = 'Medal'

In [8]:
X = teams[features]
y = teams[target]

In [9]:
# Convert the target variable to numerical using label encoding
le_medal = LabelEncoder()
y_encoded = le_medal.fit_transform(y)

In [10]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [11]:
# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [12]:
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 1


In [13]:
# Open a strategy scope.
with strategy.scope():
    # Create a simple neural network model
    model = models.Sequential([
        layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # Assuming binary classification for 'Medal'
    ])

In [14]:
 # Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
977/977 [==============================] - 5s 3ms/step - loss: 0.0666 - accuracy: 0.3377 - val_loss: -0.1179 - val_accuracy: 0.3421
Epoch 2/10
977/977 [==============================] - 1s 2ms/step - loss: -0.0010 - accuracy: 0.3378 - val_loss: -0.3133 - val_accuracy: 0.3421
Epoch 3/10
977/977 [==============================] - 1s 1ms/step - loss: -0.0358 - accuracy: 0.3378 - val_loss: -0.3881 - val_accuracy: 0.3421
Epoch 4/10
977/977 [==============================] - 1s 1ms/step - loss: -0.0782 - accuracy: 0.3378 - val_loss: -0.4443 - val_accuracy: 0.3421
Epoch 5/10
977/977 [==============================] - 1s 1ms/step - loss: -0.2360 - accuracy: 0.3377 - val_loss: -0.4083 - val_accuracy: 0.3421
Epoch 6/10
977/977 [==============================] - 1s 1ms/step - loss: -0.3740 - accuracy: 0.3377 - val_loss: -0.8143 - val_accuracy: 0.3421
Epoch 7/10
977/977 [==============================] - 1s 1ms/step - loss: -0.5442 - accuracy: 0.3378 - val_loss: -1.4721 - val_accuracy: 

In [16]:
# Evaluate the model
y_pred = model.predict(X_test)
y_pred = y_pred.round()

In [17]:
# Print classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2568
           1       0.34      1.00      0.51      2672
           2       0.00      0.00      0.00      2571

    accuracy                           0.34      7811
   macro avg       0.11      0.33      0.17      7811
weighted avg       0.12      0.34      0.17      7811



C:\Users\Shakil\miniconda3\envs\Olympic\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Shakil\miniconda3\envs\Olympic\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Shakil\miniconda3\envs\Olympic\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
# Save the model
model.save('model.h5')

In [19]:
import pickle

# Save the label encoder
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(le_medal, f)

In [20]:
import json

# Save the feature names
with open('feature_names.json', 'w') as f:
    json.dump(features, f)

In [21]:
# Save the model training history
with open('history.json', 'w') as f:
    json.dump(history.history, f)